# Navigation Project 1
Author: [Anthony Vergottis](http://github.com/anthonyvergottis) 

The aim of this project is to explore a value-based method for solving the reinforcement learning problem. The algorithic implementation is based upon a simple Deep Q Network, implementing a model-free approach. The agent interacts with a enviroment whose state space is composed of 37 continious values. There are for possible actions for the actions space:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The goal of the agent is to pick up yellow bananas (receiving a reward: +1) whilst avoiding blue bananas (receiving a reward: -1). The criteria for a good solution implementation require that the agent receives a average reward greater that 13 for 100 episodes. The simulation enviroment is provided by Unity ML.

The report is split in four parts:

1. **Solution Strategy**
2. **Hyperparameters**
3. **Results**
4. **Future work for improvements** 

## Solution Strategy

# Hyperparameters

### The hyperparameters used were as follow:

The valus are named as they are found in the code:

    - BUFFER_SIZE = int(1e5)  # replay buffer size
    - BATCH_SIZE = 64         # minibatch size
    - GAMMA = 0.99            # discount factor
    - TAU = 1e-3              # for soft update of target parameters
    - LR = 5e-4               # learning rate
    - UPDATE_EVERY = 4        # how often to update the network
    - n_episodes = 2000       # Limit of number of episodes to run
    - max_t = 1000            # Max No. of time steps per episode
    - eps_start = 1.0         # Epsilon starting value
    - eps_end = 0.01          # Smallest epsilon allowed
    - eps_decay = 0.995       # Epsilon decay rate

## Results

The DQN agent solved the environment in 449 episodes, with an Average Score: 13.03
![dqn_scores.png](images/dqn_scores.png)


## Future work for improvements

1. Implement Dual Deep Q Network
2. Try using prioritized replay memory
3. Look into the Dueling Deep Network algorithm